# Tests for the skill profiling framework

Run some tests on functions used in the skill profiling framework.

In [1]:
import pandas as pd
import numpy as np

# local imports
from config import data_dir
from analog_forecast import read_subset_era5
from run_profile import get_naive_sample_dates

Load a dataset (spatial subset) for doing some testing with.

In [2]:
%%time

varname = "t2m"
spatial_domain = "alaska"
ref_date = "2004-10-11"

raw_sub_da = read_subset_era5(spatial_domain, data_dir, varname, use_anom=False).load()

CPU times: user 4.98 s, sys: 9.69 s, total: 14.7 s
Wall time: 44.6 s


`get_naive_sample_dates` should return a tuple of ("list of naive analog dates and 14 dates for each" (`all_naive_dates` below), "list of naive analog dates" (`naive_analog_dates` below). Run some verifications.

In [3]:
all_times = raw_sub_da.time.values
all_naive_dates, naive_analog_dates = get_naive_sample_dates(all_times, ref_date)

`naive_analog_dates` should be contained within the `all_naive_dates` (ok if type is different):

In [4]:
assert np.all([dt in all_naive_dates for dt in naive_analog_dates])

`naive_analog_dates` should not contain dates that are within 15 days of forecast date:

In [5]:
ref_dt = pd.to_datetime(ref_date + " 12:00:00")
forecast_length = 14
between = pd.Series(naive_analog_dates).between(
    ref_dt - pd.to_timedelta(forecast_length + 2, "D"),
    ref_dt + pd.to_timedelta(forecast_length + 1, "D")
)
assert np.all(~between)

`naive_analog_dates` should all be within +/- 45 days of day-of-year of `ref_date`:

In [6]:
ref_day = ref_dt.day
ref_month = ref_dt.month

def get_window_from_analog_date(dt, ref_month, ref_day):
    year = pd.to_datetime(naive_analog_dates[0]).year
    tmp_ref_dt = pd.to_datetime(f"{year}-{ref_month}-{ref_day} 12:00:00")
    td_offset = pd.to_timedelta(45, "D")
    return pd.Series(tmp_ref_dt).between(
        tmp_ref_dt - (td_offset + pd.to_timedelta(1, "D")), 
        tmp_ref_dt + td_offset
    )

assert np.all([get_window_from_analog_date(dt, ref_month, ref_day) for dt in naive_analog_dates])

In [ ]:
def get_naive_sample_dates(all_times, ref_date):
    """Constructs list of all dates to be queried in some fashion for an instance of the naive forecast"""
    # limit pool of all potential times for analogs to be within 3-month window centered on
    #  day-of-year of reference date
    # iterate over all available years and use the same month-day as ref_dt to 
    #  construct acceptance window of +/- 45 days, and accumulate boolean series
    #  corresponding to all_times which essentially will be reduced via "or"
    #  operation over the year axis.
    ref_dt = pd.to_datetime(ref_date + " 12:00:00")
    td_offset = pd.to_timedelta(45, "D")
    all_times = pd.Series(all_times)
    keep_bool = []
    
    for year in np.unique(pd.Series(all_times).dt.year):
        tmp_ref_dt = pd.to_datetime(f"{year}-{ref_dt.month}-{ref_dt.day}")
        keep_bool.append(
            all_times.between(
                tmp_ref_dt - (td_offset + pd.to_timedelta(1, "D")), 
                tmp_ref_dt + td_offset
            )
        )
    keep_times = all_times[np.array(keep_bool).sum(axis=0).astype(bool)]
    
    # construct an exclusion window around ref_date, based on size of forecast which is 14 days
    forecast_length = 14
    exclude = keep_times.between(
        ref_dt - pd.to_timedelta(forecast_length + 2, "D"),
        ref_dt + pd.to_timedelta(forecast_length + 1, "D")
    )
    keep_times[~exclude]
    
    # choose 5 times at random
    analog_times = list(np.random.choice(keep_times, 5, replace=False))
    
    all_dates = []
    for t in analog_times + [ref_dt]:
        all_dates.extend(pd.date_range(t, t + pd.to_timedelta(forecast_length, "D")))
    
    return all_dates, analog_times

Results for anomaly-based and standard search options should be identical because we are randomly sampling the same windows for analogs for each, and forecasts should only be generated from raw data (i.e. not anomalies).

In [4]:
import xarray as xr
# local imports
from config import data_dir
from analog_forecast import read_subset_era5
from run_profile import profile_naive_forecast

In [6]:
varname = "t2m"
spatial_domain = "alaska"
ref_date = "2004-10-11"

In [13]:
%%time
varname = "t2m"
spatial_domain = "alaska"

anom_sub_da = read_subset_era5(spatial_domain, data_dir, varname, use_anom=True).load()
raw_sub_da = read_subset_era5(spatial_domain, data_dir, varname, use_anom=False).load()

CPU times: user 5.46 s, sys: 15.7 s, total: 21.1 s
Wall time: 56 s


In [15]:
raw_sub_da

<xarray.DataArray 't2m' (time: 23011, latitude: 129, longitude: 221)>
array([[[238.64413, 238.62225, 238.60222, ..., 238.26144, 238.28333,
         238.307  ],
        [238.59857, 238.57306, 238.54755, ..., 238.38536, 238.36896,
         238.35439],
        [238.6587 , 238.6259 , 238.58035, ..., 238.31976, 238.34892,
         238.37808],
        ...,
        [278.81815, 278.69788, 278.60858, ..., 281.8322 , 281.90692,
         281.78482],
        [278.8637 , 278.76895, 278.6924 , ..., 282.07455, 282.12195,
         282.00897],
        [278.92566, 278.83273, 278.7562 , ..., 282.09277, 282.1183 ,
         282.07275]],

       [[239.36392, 239.38397, 239.40584, ..., 241.92241, 241.97343,
         242.02446],
        [239.0924 , 239.10152, 239.107  , ..., 241.96616, 241.97162,
         241.98254],
        [238.95392, 238.95027, 238.92657, ..., 241.86227, 241.87686,
         241.88962],
...
        [279.0423 , 279.11517, 279.1917 , ..., 281.12152, 281.09964,
         280.8026 ],
        [279.1662 , 279.28284, 279.363  , ..., 281.18347, 281.11786,
         280.901  ],
        [279.46323, 279.4924 , 279.51245, ..., 280.93927, 280.86823,
         280.8755 ]],

       [[246.01161, 245.97699, 245.94783, ..., 246.29588, 245.85854,
         245.42302],
        [245.91321, 245.85307, 245.79112, ..., 245.70183, 245.28635,
         244.87086],
        [246.0098 , 245.95331, 245.90591, ..., 245.25172, 244.94922,
         244.6449 ],
        ...,
        [279.2628 , 279.39035, 279.4942 , ..., 281.0486 , 281.19077,
         280.9338 ],
        [279.3229 , 279.45776, 279.57803, ..., 281.27823, 281.40213,
         281.18164],
        [279.43225, 279.52155, 279.6163 , ..., 281.40942, 281.36386,
         281.30737]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... -125.5 -125.2 -125.0
  * latitude   (latitude) float32 76.0 75.75 75.5 75.25 ... 44.5 44.25 44.0
  * time       (time) datetime64[ns] 1959-01-01T12:00:00 ... 2021-12-31T12:00:00
Attributes:
    units:      K
    long_name:  2 metre temperature

CPU times: user 37.1 ms, sys: 6.15 ms, total: 43.3 ms
Wall time: 128 ms


In [12]:
%%time
anom_sub_da.load()

CPU times: user 107 µs, sys: 347 µs, total: 454 µs
Wall time: 465 µs


<xarray.DataArray 't2m' (time: 23011, latitude: 129, longitude: 221)>
array([[[ -8.549255  ,  -8.580078  ,  -8.612717  , ...,  -7.957306  ,
          -8.020447  ,  -8.081558  ],
        [ -8.650421  ,  -8.676575  ,  -8.702698  , ...,  -8.185608  ,
          -8.20163   ,  -8.223633  ],
        [ -8.689575  ,  -8.71756   ,  -8.749435  , ...,  -8.395996  ,
          -8.39682   ,  -8.397522  ],
        ...,
        [  0.6021118 ,   0.48345947,   0.3784485 , ...,  -0.45758057,
          -0.41134644,  -0.31118774],
        [  0.45394897,   0.3383484 ,   0.23587036, ...,  -0.33285522,
          -0.24334717,  -0.22293091],
        [  0.32632446,   0.22058105,   0.12200928, ...,  -0.3590088 ,
          -0.24658203,  -0.20925903]],

       [[ -8.701767  ,  -8.706207  ,  -8.712036  , ...,  -4.572998  ,
          -4.577545  ,  -4.5814667 ],
        [ -8.965759  ,  -8.973236  ,  -8.983871  , ...,  -4.833725  ,
          -4.807434  ,  -4.782959  ],
        [ -9.121414  ,  -9.138702  ,  -9.171005  , ...,  -5.035446  ,
          -5.0387573 ,  -5.043518  ],
...
        [  0.26367188,   0.2782898 ,   0.31729126, ...,  -1.0618286 ,
          -1.1394348 ,  -1.2284546 ],
        [  0.18429565,   0.20809937,   0.2336731 , ...,  -1.1010132 ,
          -1.1716919 ,  -1.2609253 ],
        [  0.26828003,   0.236969  ,   0.21838379, ...,  -1.3591919 ,
          -1.4099121 ,  -1.3523865 ]],

       [[ -1.248169  ,  -1.2969055 ,  -1.3448944 , ...,  -0.77874756,
          -1.2822113 ,  -1.7834015 ],
        [ -1.3666229 ,  -1.4278259 ,  -1.4911957 , ...,  -1.6291656 ,
          -2.0381165 ,  -2.4539032 ],
        [ -1.3091431 ,  -1.3617401 ,  -1.3987274 , ...,  -2.190094  ,
          -2.5091553 ,  -2.8300018 ],
        ...,
        [  0.742157  ,   0.8474426 ,   0.9310913 , ...,  -1.2331238 ,
          -1.1325073 ,  -1.1520691 ],
        [  0.64923096,   0.74450684,   0.8373718 , ...,  -1.1445923 ,
          -0.99365234,  -1.0443726 ],
        [  0.5852051 ,   0.64434814,   0.71520996, ...,  -1.0320129 ,
          -1.0059204 ,  -0.9586792 ]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 1959-01-01T12:00:00 ... 2021-12-31T12:00:00
  * latitude   (latitude) float32 76.0 75.75 75.5 75.25 ... 44.5 44.25 44.0
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... -125.5 -125.2 -125.0
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365